# Python import

In [9]:
import numpy as np
import random

# Defining the class

In [15]:
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        

    def feedForward(self, a):
        """Return the output if a is a input"""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data,                # tuple (x, y) x -> input,  y -> desired output
            epochs,                              
            mini_batch_size, 
            eta,                                # Learning rate 
            test_data=None):
        if test_data:
            n_test = len(test_data)
            
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batches(mini_batch, eta)
                
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test) )
            else:
                print ("Epoch {0} complete".format(j) )
    
    
    def update_mini_batches(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x,y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta/len(mini_batch)) * nw 
                           for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/ len(mini_batch)) * nb
                           for b, nb in zip(self.biases, nabla_b)]
        
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedForward(x)), y) for(x, y) in test_data]
        
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
        

In [7]:
# test the Network structure
n = Network([2, 3, 1])

print("Sizes : ")
print(n.sizes)
print("Biases: ")
print(n.biases)
print("Weights: ")
print(n.weights)

Sizes : 
[2, 3, 1]
Biases: 
[array([[ 0.03616646],
       [-1.04239321],
       [ 1.18799311]]), array([[ 0.53542047]])]
Weights: 
[array([[-1.17647372, -0.10428803],
       [-0.09355442,  0.85018297],
       [ 0.99908164, -1.13188315]]), array([[-1.12402966, -1.31487048,  0.36691948]])]


# Implementing network helper functions

In [10]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    """Deriavtive of the sigmoid function"""
    return sigmoid(z) * (1 - sigmoid(z))


# Loading data and training the model

In [22]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

FileNotFoundError: [Errno 2] No such file or directory: 'mnist.pkl.gz'